# Practical 4: National Environmental Footprints

In [1]:
# Import modules
import pandas as pd
import numpy as np

# Set pandas number format
pd.options.display.float_format = '{:,.2f}'.format

## Data pre-processing

### Load data

In [2]:
dir_path = "../data/nat_env_footprint/"

The folder has the following structure:  

```
├── labels  
│   ├── labels.csv  
│   ├── multi_reg_final_demand.csv  
│   └── multi_reg_sectors.csv  
├── F_y.txt  
├── F.txt  
├── pop.txt  
├── V.txt  
├── Y.txt  
└── Z.txt  
```

In [3]:
# Import Z, Y, V, F and population data
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=None)
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter='\t', header=None)
V = pd.read_csv(f"{dir_path}V.txt", delimiter='\t', header=None)
F = pd.read_csv(f"{dir_path}F.txt", delimiter='\t', header=None)
F_y = pd.read_csv(f"{dir_path}F_y.txt", delimiter='\t', header=None)

In [4]:
# Import the labels.csv file from the data folder 
labels = pd.read_csv(f"{dir_path}labels/labels.csv", delimiter=",")
labels

,region_code,region_name,sector_code,sector_category,final_demand_code,final_demand_category,value_added_code,value_added_category,extension_code,extension_name
0,R1,OECD,S1,Food,F1,Final consumption expenditure by household,V1,value_added,E1,CO2 emissions (unit: tonnes/year)
1,R2,BRICS,S2,Clothing,F2,Final consumption expenditure by NPISHs,NaN,NaN,E2,Blue water consumption (unit: million m3/year)
2,R3,ROW,S3,Shelter,F3,Final consumption expenditure by government,NaN,NaN,E3,Employment (unit: 1000 people/year)
3,NaN,NaN,S4,Construction,F4,Gross capital formation,NaN,NaN,NaN,NaN
4,NaN,NaN,S5,Manufactured products,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,S6,Mobility,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,S7,Trade,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,S8,Services,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Get sector labels (in './labels/multi_reg_sectors.csv')
region_and_sector_labels = pd.read_csv(f"{dir_path}labels/multi_reg_sectors.csv")
region_and_sector_labels = pd.MultiIndex.from_frame(region_and_sector_labels)

# Get Y labels (in './labels/multi_reg_sectors.csv')
y_labels = pd.read_csv(f"{dir_path}labels/multi_reg_final_demand.csv")
y_labels = pd.MultiIndex.from_frame(y_labels)

# Get labels for V and F (in './labels/labels.csv')
region_labels = labels['region_name'].dropna()  # series --> not need for multiindex
value_added_labels = labels['value_added_category'].dropna()  # series --> not need for multiindex
extension_labels = labels['extension_name'].dropna()  # series --> not need for multiindex

In [6]:
# Visualize
region_and_sector_labels
value_added_labels
extension_labels
y_labels

MultiIndex([( 'OECD',  'Final consumption expenditure by household'),
            ( 'OECD',     'Final consumption expenditure by NPISHs'),
            ( 'OECD', 'Final consumption expenditure by government'),
            ( 'OECD',                     'Gross capital formation'),
            ('BRICS',  'Final consumption expenditure by household'),
            ('BRICS',     'Final consumption expenditure by NPISHs'),
            ('BRICS', 'Final consumption expenditure by government'),
            ('BRICS',                     'Gross capital formation'),
            (  'ROW',  'Final consumption expenditure by household'),
            (  'ROW',     'Final consumption expenditure by NPISHs'),
            (  'ROW', 'Final consumption expenditure by government'),
            (  'ROW',                     'Gross capital formation')],
           names=['region', 'final_demand_category'])

### Update labels

In [7]:
# Set columns and indexes for each dataframe

# for Z
Z.index = region_and_sector_labels
Z.columns = region_and_sector_labels

# For final demand
Y.index = region_and_sector_labels
Y.columns = y_labels

# For value added
V.index =  value_added_labels  # value_added_labels is a series, not a frame
V.columns = region_and_sector_labels

# For extensions
F.index = extension_labels  # extension_labels is a series, not a frame
F.columns = region_and_sector_labels

# For final demand extension
F_y.index = extension_labels  # extension_labels is a series, not a frame
F_y.columns = region_labels

In [8]:
# Visualize
Y  # Z, F, V, F_y

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                1,753,360.00   
       Clothing                                              288,486.92   
       Shelter                                               662,214.58   
       Construction                                           68,488.30   
       Manufactured products                               1,786,986.30   
       Mobility                                            1,157,829.50   
       Trade                                                 400,433.69   
       Services                                           10,700,678.00   
BRICS  Food                                                   22,234.31   
       Clothing                                              107,713.19   
       Shelter                                                14,536.06   
       Construction                                            3,443.30   
       Manufactured products                                 164,403.66   
       Mobility                                               19,298.19   
       Trade                                                     182.89   
       Services                                               22,002.01   
ROW    Food                                                  136,101.71   
       Clothing                                               76,581.74   
       Shelter                                                26,597.90   
       Construction                                            5,852.93   
       Manufactured products                                 282,931.03   
       Mobility                                               48,716.57   
       Trade                                                     544.82   
       Services                                               81,667.93   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                 1,610.14   
       Clothing                                                12.91   
       Shelter                                              1,939.06   
       Construction                                           134.97   
       Manufactured products                                4,629.97   
       Mobility                                             9,812.75   
       Trade                                                2,697.26   
       Services                                         2,968,987.10   
BRICS  Food                                                    14.16   
       Clothing                                                 0.00   
       Shelter                                                  2.45   
       Construction                                             0.01   
       Manufactured products                                  402.37   
       Mobility                                                24.89   
       Trade                                                  312.41   
       Services                                               977.11   
ROW    Food                                                   106.58   
       Clothing                                                 0.01   
       Shelter                                                 24.82   
       Construction                                             0.17   
       Manufactured products                                2,148.11   
       Mobility                                               246.04   
       Trade                                                  921.90   
       Services                                            22,460.62   

region                       

## Input-Output calculations

### total product intputs and outputs

$\mathbf{x} = \mathbf{Ax} + \mathbf{Y} = \mathbf{LY}$

In [9]:
# Compute total production inputs and outputs
x_in = Z.sum(axis=1) + Y.sum(axis=1)
x_out = Z.sum(axis=0) + V.sum(axis=0)

# Sanity check
if not np.allclose(x_in, x_out):
    raise ValueError(f"discrepancies between\nx_in\n{x_in}\n\nx_out\n{x_out}")

x = x_out

### Leontief inverse of quantity model

In [10]:
# inverse X
inv_diag_x = np.linalg.inv(np.diag(x))

# A (Unit: Million euro/Million euro)
A = Z @ inv_diag_x  # @ = np.matmul
A.columns = region_and_sector_labels  
# alternative :
# A = Z.divide(X)  # then no need for set columns

# Create an identity matrix the same order
Id = np.identity(len(region_and_sector_labels))

# Compute leontief inverse matrix in the Demand-pull model
L_values = np.linalg.inv((Id - A))
L = pd.DataFrame(L_values, index=region_and_sector_labels, columns=region_and_sector_labels)

# we also save LY to avoid repeating calculations
LY = L @ Y

In [11]:
# Sanity check
# ! add '.sum(axis=1)' since Y is dataframe
if not np.allclose(x, LY.sum(axis=1)):
    raise ValueError("Discrepancy with x")

### Modified final demand calculation

#### Mathematical approach
##### Select a region

In [12]:
# 1 Make vector of zeros of the same length of the columns of Y
i_j = np.zeros(len(Y.columns))
# 2 Turn into 1's the 0s of the columns you don't want to analyse
i_j[:4] = 1
# 3 multiply the diagonlized i_j vector by Y 
Y_oecd =  Y @ np.diag(i_j)
Y_oecd.columns = y_labels
Y_oecd

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                1,753,360.00   
       Clothing                                              288,486.92   
       Shelter                                               662,214.58   
       Construction                                           68,488.30   
       Manufactured products                               1,786,986.30   
       Mobility                                            1,157,829.50   
       Trade                                                 400,433.69   
       Services                                           10,700,678.00   
BRICS  Food                                                   22,234.31   
       Clothing                                              107,713.19   
       Shelter                                                14,536.06   
       Construction                                            3,443.30   
       Manufactured products                                 164,403.66   
       Mobility                                               19,298.19   
       Trade                                                     182.89   
       Services                                               22,002.01   
ROW    Food                                                  136,101.71   
       Clothing                                               76,581.74   
       Shelter                                                26,597.90   
       Construction                                            5,852.93   
       Manufactured products                                 282,931.03   
       Mobility                                               48,716.57   
       Trade                                                     544.82   
       Services                                               81,667.93   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                 1,610.14   
       Clothing                                                12.91   
       Shelter                                              1,939.06   
       Construction                                           134.97   
       Manufactured products                                4,629.97   
       Mobility                                             9,812.75   
       Trade                                                2,697.26   
       Services                                         2,968,987.10   
BRICS  Food                                                    14.16   
       Clothing                                                 0.00   
       Shelter                                                  2.45   
       Construction                                             0.01   
       Manufactured products                                  402.37   
       Mobility                                                24.89   
       Trade                                                  312.41   
       Services                                               977.11   
ROW    Food                                                   106.58   
       Clothing                                                 0.01   
       Shelter                                                 24.82   
       Construction                                             0.17   
       Manufactured products                                2,148.11   
       Mobility                                               246.04   
       Trade                                                  921.90   
       Services                                            22,460.62   

region                       

##### Select a sector within a region

As example, we will look for '`gross fixed capital formation`' for the '`BRICS`' region

In [13]:
# find the region position
index_lvl_1_position = 1

# find column position of the category among all categories
index_lvl_2_position = 3  # out of 4 categories
nb_y_categories = 4

# calculate the column we are interested in
position = (index_lvl_1_position*nb_y_categories) + index_lvl_2_position

# and repeat the process
i_j = np.zeros(len(Y.columns))
i_j[position] = 1
Y_brics_gfcf = Y @ np.diag(i_j)
Y_brics_gfcf.columns = y_labels
Y_brics_gfcf

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
BRICS  Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
ROW    Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
BRICS  Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
ROW    Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   

region                       

##### Apply change to specific product consumed in a specific region

For example, check a 20% change in the product '`Shelter`' consummed in '`OECD`'

In [14]:
CHANGE = 0.2 # 1

# For the columns 
col_position = range(4)
cols_i_j = np.zeros(len(Y.columns))
cols_i_j[:4] = 1  # we want all 4 categories of OECD region

# For the rows
rows_i_j = np.zeros(len(Y.index))  # replace columns <-> index
rows_i_j[2] = CHANGE  # we want 20% (=CHANGE) of shelter only (index = 2)

# compute oecd x shelter
Y_oecd_shelter = np.diag(rows_i_j) @ Y @ np.diag(cols_i_j)
Y_oecd_shelter.columns = y_labels
Y_oecd_shelter.index = region_and_sector_labels
Y_oecd_shelter

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                               132,442.92   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
BRICS  Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
ROW    Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                387.81   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
BRICS  Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
ROW    Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   

region                       

#### Slicing approach
##### Select a region

In [15]:
# Select data at the 1st level of either index or columns (i.e. Region)
Y.loc["OECD", :]  # for the rows
Y.loc[:, "OECD"]  # for the columns

final_demand_category         Final consumption expenditure by household  \
region sector                                                              
OECD   Food                                                 1,753,360.00   
       Clothing                                               288,486.92   
       Shelter                                                662,214.58   
       Construction                                            68,488.30   
       Manufactured products                                1,786,986.30   
       Mobility                                             1,157,829.50   
       Trade                                                  400,433.69   
       Services                                            10,700,678.00   
BRICS  Food                                                    22,234.31   
       Clothing                                               107,713.19   
       Shelter                                                 14,536.06   
       Construction                                             3,443.30   
       Manufactured products                                  164,403.66   
       Mobility                                                19,298.19   
       Trade                                                      182.89   
       Services                                                22,002.01   
ROW    Food                                                   136,101.71   
       Clothing                                                76,581.74   
       Shelter                                                 26,597.90   
       Construction                                             5,852.93   
       Manufactured products                                  282,931.03   
       Mobility                                                48,716.57   
       Trade                                                      544.82   
       Services                                                81,667.93   

final_demand_category         Final consumption expenditure by NPISHs  \
region sector                                                           
OECD   Food                                                  1,610.14   
       Clothing                                                 12.91   
       Shelter                                               1,939.06   
       Construction                                            134.97   
       Manufactured products                                 4,629.97   
       Mobility                                              9,812.75   
       Trade                                                 2,697.26   
       Services                                          2,968,987.10   
BRICS  Food                                                     14.16   
       Clothing                                                  0.00   
       Shelter                                                   2.45   
       Construction                                              0.01   
       Manufactured products                                   402.37   
       Mobility                                                 24.89   
       Trade                                                   312.41   
       Services                                                977.11   
ROW    Food                                                    106.58   
       Clothing                                                  0.01   
       Shelter                                                  24.82   
       Construction                                              0.17   
       Manufactured products                                 2,148.11   
       Mobility                                                246.04   
       Trade                                                   921.90   
       Services                                             22,460.62   

final_demand_category         Final consumption expenditure by government  \
region sector                                  

In [16]:
# to preserve original shape, we can do this

# create empty DataFrame with the right shape, and fill only with relevant values
Y_oecd = pd.DataFrame(
    data=0,
    index=region_and_sector_labels,
    columns=Y.columns
)

Y_oecd.loc[:, "OECD"] = Y.loc[:, "OECD"].values
Y_oecd

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                1,753,360.00   
       Clothing                                              288,486.92   
       Shelter                                               662,214.58   
       Construction                                           68,488.30   
       Manufactured products                               1,786,986.30   
       Mobility                                            1,157,829.50   
       Trade                                                 400,433.69   
       Services                                           10,700,678.00   
BRICS  Food                                                   22,234.31   
       Clothing                                              107,713.19   
       Shelter                                                14,536.06   
       Construction                                            3,443.30   
       Manufactured products                                 164,403.66   
       Mobility                                               19,298.19   
       Trade                                                     182.89   
       Services                                               22,002.01   
ROW    Food                                                  136,101.71   
       Clothing                                               76,581.74   
       Shelter                                                26,597.90   
       Construction                                            5,852.93   
       Manufactured products                                 282,931.03   
       Mobility                                               48,716.57   
       Trade                                                     544.82   
       Services                                               81,667.93   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                 1,610.14   
       Clothing                                                12.91   
       Shelter                                              1,939.06   
       Construction                                           134.97   
       Manufactured products                                4,629.97   
       Mobility                                             9,812.75   
       Trade                                                2,697.26   
       Services                                         2,968,987.10   
BRICS  Food                                                    14.16   
       Clothing                                                 0.00   
       Shelter                                                  2.45   
       Construction                                             0.01   
       Manufactured products                                  402.37   
       Mobility                                                24.89   
       Trade                                                  312.41   
       Services                                               977.11   
ROW    Food                                                   106.58   
       Clothing                                                 0.01   
       Shelter                                                 24.82   
       Construction                                             0.17   
       Manufactured products                                2,148.11   
       Mobility                                               246.04   
       Trade                                                  921.90   
       Services                                            22,460.62   

region                       

##### Select a sector within a region

As example, we will look for '`gross fixed capital formation`' for the '`BRICS`' region

In [17]:
# Select "Gross capital formation" for BRICS region
# ! .loc[] alone does NOT work for 2nd + level of multiindex, instead we do:
column_slice = pd.IndexSlice['BRICS', "Gross capital formation"]
Y.loc[:, column_slice]

region  sector               
OECD    Food                        2,303.22
        Clothing                      160.53
        Shelter                       247.89
        Construction                5,940.46
        Manufactured products     202,829.52
        Mobility                      384.03
        Trade                       1,899.61
        Services                    3,957.35
BRICS   Food                       77,952.18
        Clothing                   18,381.93
        Shelter                    76,181.84
        Construction            2,474,604.30
        Manufactured products   1,728,212.10
        Mobility                   23,825.46
        Trade                      14,021.23
        Services                  351,848.15
ROW     Food                        2,982.71
        Clothing                      153.46
        Shelter                     1,679.75
        Construction                7,029.51
        Manufactured products      92,569.03
        Mobility         

In [18]:
# Select "Gross capital formation" for every region
Y.loc[:, pd.IndexSlice[:, "Gross capital formation"]]

region                                          OECD                   BRICS  \
final_demand_category        Gross capital formation Gross capital formation   
region sector                                                                  
OECD   Food                                45,566.90                2,303.22   
       Clothing                            12,133.65                  160.53   
       Shelter                            117,857.03                  247.89   
       Construction                     3,064,119.50                5,940.46   
       Manufactured products            2,092,678.30              202,829.52   
       Mobility                            27,611.03                  384.03   
       Trade                               59,792.34                1,899.61   
       Services                         1,191,428.10                3,957.35   
BRICS  Food                                   642.14               77,952.18   
       Clothing                             1,020.32               18,381.93   
       Shelter                              2,315.73               76,181.84   
       Construction                         1,893.45            2,474,604.30   
       Manufactured products              173,872.81            1,728,212.10   
       Mobility                               889.99               23,825.46   
       Trade                                   56.21               14,021.23   
       Services                             4,325.40              351,848.15   
ROW    Food                                 2,170.23                2,982.71   
       Clothing                               807.07                  153.46   
       Shelter                              9,482.55                1,679.75   
       Construction                         5,776.29                7,029.51   
       Manufactured products              237,441.72               92,569.03   
       Mobility                               931.51                  584.08   
       Trade                                  133.68                  217.97   
       Services                            12,671.66                1,527.35   

region                                           ROW  
final_demand_category        Gross capital formation  
region sector                                         
OECD   Food                                 1,864.09  
       Clothing                             1,310.92  
       Shelter                              1,421.05  
       Construction                        12,952.62  
       Manufactured products              304,226.85  
       Mobility                             2,045.06  
       Trade                                3,657.03  
       Services                            31,172.16  
BRICS  Food                                   466.37  
       Clothing                             2,013.96  
       Shelter                              2,712.00  
       Construction                         4,056.56  
       Manufactured products              179,023.86  
       Mobility                               233.44  
       Trade                                  816.20  
       Services                             5,585.34  
ROW    Food                                30,615.44  
       Clothing                             8,833.76  
       Shelter                             31,517.67  
       Construction                     1,960,927.20  
       Manufactured products              812,728.75  
       Mobility                            47,383.45  
       Trade                               59,164.84  
       Services                           322,754.19

In [19]:
# Similarly, to preserve original shape, we can do this

# create empty DataFrame with the right shape, and fill only with relevant values
Y_brics_gfcf = pd.DataFrame(
    data=0,
    index=region_and_sector_labels,
    columns=Y.columns
)

Y_brics_gfcf.loc[:, column_slice] = Y.loc[:, column_slice].values
Y_brics_gfcf

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                           0   
       Clothing                                                       0   
       Shelter                                                        0   
       Construction                                                   0   
       Manufactured products                                          0   
       Mobility                                                       0   
       Trade                                                          0   
       Services                                                       0   
BRICS  Food                                                           0   
       Clothing                                                       0   
       Shelter                                                        0   
       Construction                                                   0   
       Manufactured products                                          0   
       Mobility                                                       0   
       Trade                                                          0   
       Services                                                       0   
ROW    Food                                                           0   
       Clothing                                                       0   
       Shelter                                                        0   
       Construction                                                   0   
       Manufactured products                                          0   
       Mobility                                                       0   
       Trade                                                          0   
       Services                                                       0   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                        0   
       Clothing                                                    0   
       Shelter                                                     0   
       Construction                                                0   
       Manufactured products                                       0   
       Mobility                                                    0   
       Trade                                                       0   
       Services                                                    0   
BRICS  Food                                                        0   
       Clothing                                                    0   
       Shelter                                                     0   
       Construction                                                0   
       Manufactured products                                       0   
       Mobility                                                    0   
       Trade                                                       0   
       Services                                                    0   
ROW    Food                                                        0   
       Clothing                                                    0   
       Shelter                                                     0   
       Construction                                                0   
       Manufactured products                                       0   
       Mobility                                                    0   
       Trade                                                       0   
       Services                                                    0   

region                       

##### Apply change to specific product consumed in a specific region

For example, make a 20% change in the product '`Shelter`' consummed in '`OECD`'

In [21]:
# Select the slice
row_slice = pd.IndexSlice['OECD', "Shelter"]

# create empty DataFrame with the right shape, and fill only with relevant values
Y_oecd_shelter = pd.DataFrame(
    data=0,
    index=region_and_sector_labels,
    columns=Y.columns
)

Y_oecd_shelter.loc[row_slice, "OECD"] = 0.2 * Y.loc[row_slice, "OECD"].values
Y_oecd_shelter

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                               132,442.92   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
BRICS  Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   
ROW    Food                                                        0.00   
       Clothing                                                    0.00   
       Shelter                                                     0.00   
       Construction                                                0.00   
       Manufactured products                                       0.00   
       Mobility                                                    0.00   
       Trade                                                       0.00   
       Services                                                    0.00   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                387.81   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
BRICS  Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   
ROW    Food                                                     0.00   
       Clothing                                                 0.00   
       Shelter                                                  0.00   
       Construction                                             0.00   
       Manufactured products                                    0.00   
       Mobility                                                 0.00   
       Trade                                                    0.00   
       Services                                                 0.00   

region                       

## Carbon footprints

In [22]:
# visualise extension for final demand
F_y

region_name,OECD,BRICS,ROW
extension_name,,,
CO2 emissions (unit: tonnes/year),"2,643,610,400.00","1,057,966,300.00","1,415,817,700.00"
Blue water consumption (unit: million m3/year),"13,479.62","26,402.66","33,093.16"
Employment (unit: 1000 people/year),0.00,0.00,0.00


In [23]:
# Visualize extensions per sector x region
F

region                                                   OECD                \
sector                                                   Food      Clothing   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              398,725,000.00 55,971,646.00   
Blue water consumption (unit: million m3/year)     156,813.04        490.17   
Employment (unit: 1000 people/year)                 22,408.17      3,788.49   

region                                                           \
sector                                                  Shelter   
extension_name                                                    
CO2 emissions (unit: tonnes/year)              4,138,056,600.00   
Blue water consumption (unit: million m3/year)         6,443.69   
Employment (unit: 1000 people/year)                   13,277.17   

region                                                         \
sector                                           Construction   
extension_name                                                  
CO2 emissions (unit: tonnes/year)              653,931,880.00   
Blue water consumption (unit: million m3/year)         694.02   
Employment (unit: 1000 people/year)                 34,733.99   

region                                                                \
sector                                         Manufactured products   
extension_name                                                         
CO2 emissions (unit: tonnes/year)                     941,505,400.00   
Blue water consumption (unit: million m3/year)              6,113.67   
Employment (unit: 1000 people/year)                        41,141.60   

region                                                                         \
sector                                                 Mobility         Trade   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)              1,476,097,800.00 57,222,466.00   
Blue water consumption (unit: million m3/year)             0.00          0.00   
Employment (unit: 1000 people/year)                   15,852.87     52,900.93   

region                                                                 BRICS  \
sector                                               Services           Food   
extension_name                                                                 
CO2 emissions (unit: tonnes/year)              636,312,260.00 371,074,080.00   
Blue water consumption (unit: million m3/year)           0.00     487,328.26   
Employment (unit: 1000 people/year)                260,460.86     484,657.61   

region                                                        ...  \
sector                                              Clothing  ...   
extension_name                                                ...   
CO2 emissions (unit: tonnes/year)              66,653,270.00  ...   
Blue water consumption (unit: million m3/year)      1,869.05  ...   
Employment (unit: 1000 people/year)                33,920.57  ...   

region                                                                       \
sector                                                 Trade       Services   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              30,088,704.00 465,015,530.00   
Blue water consumption (unit: million m3/year)          0.00           0.00   
Employment (unit: 1000 people/year)                67,105.29     281,657.74   

region                                                    ROW                 \
sector                                                   Food       Clothing   
extension_name                                                                 
CO2 emissions (unit: tonnes/year)              395,165,850.00 227,249,710.00   
Blue water consumption (unit: million m3/year)     363,665.79       3,002.42   
Employment (unit: 1000 people/year)     

### CO2 intensity

$\mathbf{f} = \mathbf{F} \hat{\mathbf{x}}^{-1} $

In [24]:
# Extensions intensity vector
f = F @ inv_diag_x
f.columns = region_and_sector_labels
f

region                                           OECD                    \
sector                                           Food Clothing  Shelter   
extension_name                                                            
CO2 emissions (unit: tonnes/year)              119.59   105.70 1,680.34   
Blue water consumption (unit: million m3/year)   0.05     0.00     0.00   
Employment (unit: 1000 people/year)              0.01     0.01     0.01   

region                                                       \
sector                                         Construction   
extension_name                                                
CO2 emissions (unit: tonnes/year)                    137.56   
Blue water consumption (unit: million m3/year)         0.00   
Employment (unit: 1000 people/year)                    0.01   

region                                                                         \
sector                                         Manufactured products Mobility   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                              82.66   368.56   
Blue water consumption (unit: million m3/year)                  0.00     0.00   
Employment (unit: 1000 people/year)                             0.00     0.00   

region                                                         BRICS           \
sector                                         Trade Services   Food Clothing   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)              22.13    19.09 127.88    52.16   
Blue water consumption (unit: million m3/year)  0.00     0.00   0.17     0.00   
Employment (unit: 1000 people/year)             0.02     0.01   0.17     0.03   

region                                          ...                   ROW  \
sector                                          ... Trade Services   Food   
extension_name                                  ...                         
CO2 emissions (unit: tonnes/year)               ... 30.96    57.23 157.52   
Blue water consumption (unit: million m3/year)  ...  0.00     0.00   0.14   
Employment (unit: 1000 people/year)             ...  0.07     0.03   0.21   

region                                                                         \
sector                                         Clothing  Shelter Construction   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                624.63 2,288.24       367.14   
Blue water consumption (unit: million m3/year)     0.01     0.00         0.00   
Employment (unit: 1000 people/year)                0.05     0.05         0.05   

region                                                                         \
sector                                         Manufactured products Mobility   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                             153.04   421.92   
Blue water consumption (unit: million m3/year)                  0.00     0.00   
Employment (unit: 1000 people/year)                             0.03     0.03   

region                                                         
sector                                         Trade Services  
extension_name                                                 
CO2 emissions (unit: tonnes/year)              55.28    50.16  
Blue water consumption (unit: million m3/year)  0.00     0.00  
Employment (unit: 1000 people/year)             0.08     0.04  

[3 rows x 24 columns]

In [25]:
# This is also equivalent
f = F / x.T  # .T = .transpose()
# ! but we should always check for nan values or inf values since 1/0 = inf
f = f.replace([np.inf, -np.inf, np.nan], 0)
f

region                                           OECD                    \
sector                                           Food Clothing  Shelter   
extension_name                                                            
CO2 emissions (unit: tonnes/year)              119.59   105.70 1,680.34   
Blue water consumption (unit: million m3/year)   0.05     0.00     0.00   
Employment (unit: 1000 people/year)              0.01     0.01     0.01   

region                                                       \
sector                                         Construction   
extension_name                                                
CO2 emissions (unit: tonnes/year)                    137.56   
Blue water consumption (unit: million m3/year)         0.00   
Employment (unit: 1000 people/year)                    0.01   

region                                                                         \
sector                                         Manufactured products Mobility   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                              82.66   368.56   
Blue water consumption (unit: million m3/year)                  0.00     0.00   
Employment (unit: 1000 people/year)                             0.00     0.00   

region                                                         BRICS           \
sector                                         Trade Services   Food Clothing   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)              22.13    19.09 127.88    52.16   
Blue water consumption (unit: million m3/year)  0.00     0.00   0.17     0.00   
Employment (unit: 1000 people/year)             0.02     0.01   0.17     0.03   

region                                          ...                   ROW  \
sector                                          ... Trade Services   Food   
extension_name                                  ...                         
CO2 emissions (unit: tonnes/year)               ... 30.96    57.23 157.52   
Blue water consumption (unit: million m3/year)  ...  0.00     0.00   0.14   
Employment (unit: 1000 people/year)             ...  0.07     0.03   0.21   

region                                                                         \
sector                                         Clothing  Shelter Construction   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                624.63 2,288.24       367.14   
Blue water consumption (unit: million m3/year)     0.01     0.00         0.00   
Employment (unit: 1000 people/year)                0.05     0.05         0.05   

region                                                                         \
sector                                         Manufactured products Mobility   
extension_name                                                                  
CO2 emissions (unit: tonnes/year)                             153.04   421.92   
Blue water consumption (unit: million m3/year)                  0.00     0.00   
Employment (unit: 1000 people/year)                             0.03     0.03   

region                                                         
sector                                         Trade Services  
extension_name                                                 
CO2 emissions (unit: tonnes/year)              55.28    50.16  
Blue water consumption (unit: million m3/year)  0.00     0.00  
Employment (unit: 1000 people/year)             0.08     0.04  

[3 rows x 24 columns]

### CO2 footprint per region and final demand category

$\mathbf{e} = \mathbf{fLY}$  

In [26]:
# only keep relevant index
CO2 = 'CO2 emissions (unit: tonnes/year)'
f_co2 = f.loc[CO2]
# Calculate footprint
fLY_co2 = f_co2 @ LY  # LY = L @ Y
fLY_co2

region  final_demand_category                      
OECD    Final consumption expenditure by household    6,195,669,577.43
        Final consumption expenditure by NPISHs         410,501,128.55
        Final consumption expenditure by government     945,235,331.60
        Gross capital formation                       2,940,608,644.04
BRICS   Final consumption expenditure by household    3,960,358,509.29
        Final consumption expenditure by NPISHs         685,406,224.59
        Final consumption expenditure by government   1,348,960,727.60
        Gross capital formation                       6,306,991,692.41
ROW     Final consumption expenditure by household    3,321,607,350.15
        Final consumption expenditure by NPISHs         314,553,685.28
        Final consumption expenditure by government     621,535,598.75
        Gross capital formation                       2,589,339,265.33
Name: CO2 emissions (unit: tonnes/year), dtype: float64

### Global CO2 footprint

$\mathbf{e}_{\text{footprint}} = \mathbf{fLY} + \mathbf{F}_{y}$

In [27]:
# compute total GLOBAL footprint
F_y_co2 = F_y.loc[CO2]
fLY_co2.sum(axis=0) + F_y_co2.sum(axis=0)

34758162135.0

### CO2 footprint of a specific region

In [28]:
# To select a specific region
region = 'OECD'
fLY_co2.loc[region].sum(axis=0) + F_y_co2.loc[region].sum(axis=0)

13135625081.616154

### CO2 footprint of every region

In [29]:
# use groupby().sum() to show every region
fLY_co2.groupby('region').sum() + F_y_co2  # F_y is already aggregated by region

BRICS   13,359,683,453.88
OECD    13,135,625,081.62
ROW      8,262,853,599.50
Name: CO2 emissions (unit: tonnes/year), dtype: float64

### Sectoral contribution to CO2 footprint of a region

$\mathbf{F}_s = \hat{\mathbf{f}} \mathbf{L} \mathbf{Y}$ applied to '`BRICS`' region

In [30]:
# Select all final demand for 'BRICS' region only
Y_brics = Y.loc[:, "BRICS"].sum(axis=1)

diagfLY_co2_brics = np.diag(f_co2) @ L @ Y_brics
diagfLY_co2_brics.index = region_and_sector_labels
diagfLY_co2_brics

region  sector               
OECD    Food                       13,455,962.10
        Clothing                    2,303,451.80
        Shelter                   151,816,923.10
        Construction                9,973,621.37
        Manufactured products      76,411,303.88
        Mobility                   59,101,594.12
        Trade                       2,803,609.70
        Services                   11,051,575.01
BRICS   Food                      320,602,092.64
        Clothing                   40,804,311.97
        Shelter                 5,546,749,573.79
        Construction            2,261,010,412.06
        Manufactured products   1,957,439,674.97
        Mobility                  567,872,900.90
        Trade                      25,153,561.29
        Services                  429,664,382.03
ROW     Food                       31,455,485.05
        Clothing                   12,875,544.11
        Shelter                   556,389,702.25
        Construction               50,4

In [31]:
# which is equivalent to 
LY_brics = LY.loc[:, 'BRICS'].sum(axis=1)
diagfLY_co2_brics_ = np.diag(f_co2) @ LY_brics

if not np.allclose(diagfLY_co2_brics_, diagfLY_co2_brics):
    raise ValueError("fLY are different")

# The only difference is that it returns a numpy array instead of a pd.Series, so we need the exrta step:
diagfLY_co2_brics = pd.Series(
    data=diagfLY_co2_brics_,
    index=region_and_sector_labels
)

## Going further
### `np.dot()` vs `np.multiply()`

- The use of `np.multiply()` enables vectorization (~parallelization)
- `*` = `np.multiply()`
- `@` = `np.dot()`

In [32]:
# Applying diag() and .dot()
np.diag(f_co2) @ LY_brics

# is equivalent to .multiply() between a vector and a matrix (but it's straight in a pd.Series)
f_co2 * LY_brics

region  sector               
OECD    Food                       13,455,962.10
        Clothing                    2,303,451.80
        Shelter                   151,816,923.10
        Construction                9,973,621.37
        Manufactured products      76,411,303.88
        Mobility                   59,101,594.12
        Trade                       2,803,609.70
        Services                   11,051,575.01
BRICS   Food                      320,602,092.64
        Clothing                   40,804,311.97
        Shelter                 5,546,749,573.79
        Construction            2,261,010,412.06
        Manufactured products   1,957,439,674.97
        Mobility                  567,872,900.90
        Trade                      25,153,561.29
        Services                  429,664,382.03
ROW     Food                       31,455,485.05
        Clothing                   12,875,544.11
        Shelter                   556,389,702.25
        Construction               50,4

In [33]:
# the advantage is that we can now do several vectors at once (e.g. all f extensions)
f * LY_brics

region                                                  OECD               \
sector                                                  Food     Clothing   
extension_name                                                              
CO2 emissions (unit: tonnes/year)              13,455,962.10 2,303,451.80   
Blue water consumption (unit: million m3/year)      5,292.04        20.17   
Employment (unit: 1000 people/year)                   756.22       155.91   

region                                                                      \
sector                                                Shelter Construction   
extension_name                                                               
CO2 emissions (unit: tonnes/year)              151,816,923.10 9,973,621.37   
Blue water consumption (unit: million m3/year)         236.41        10.58   
Employment (unit: 1000 people/year)                    487.11       529.75   

region                                                                \
sector                                         Manufactured products   
extension_name                                                         
CO2 emissions (unit: tonnes/year)                      76,411,303.88   
Blue water consumption (unit: million m3/year)                496.18   
Employment (unit: 1000 people/year)                         3,339.00   

region                                                                     \
sector                                              Mobility        Trade   
extension_name                                                              
CO2 emissions (unit: tonnes/year)              59,101,594.12 2,803,609.70   
Blue water consumption (unit: million m3/year)          0.00         0.00   
Employment (unit: 1000 people/year)                   634.73     2,591.88   

region                                                                BRICS  \
sector                                              Services           Food   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              11,051,575.01 320,602,092.64   
Blue water consumption (unit: million m3/year)          0.00     421,043.85   
Employment (unit: 1000 people/year)                 4,523.73     418,736.45   

region                                                        ...  \
sector                                              Clothing  ...   
extension_name                                                ...   
CO2 emissions (unit: tonnes/year)              40,804,311.97  ...   
Blue water consumption (unit: million m3/year)      1,144.21  ...   
Employment (unit: 1000 people/year)                20,765.75  ...   

region                                                                       \
sector                                                 Trade       Services   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              25,153,561.29 429,664,382.03   
Blue water consumption (unit: million m3/year)          0.00           0.00   
Employment (unit: 1000 people/year)                56,098.69     260,245.71   

region                                                   ROW                \
sector                                                  Food      Clothing   
extension_name                                                               
CO2 emissions (unit: tonnes/year)              31,455,485.05 12,875,544.11   
Blue water consumption (unit: million m3/year)     28,948.06        170.11   
Employment (unit: 1000 people/year)                41,678.20      1,041.91   

region                                                                       \
sector                                                Shelter  Construction   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              556,389,702.25 50,490,479.26   
Blue water consumption

In [34]:
# whereas the following does not work
np.diag(f) @ LY_brics

Exception: Dot product shape mismatch, (24,) vs (3,)

In [35]:
# Instead we would have had to do this

result = pd.DataFrame(
    data=np.nan,
    index=f.index,
    columns=Y.index
)

for indicator in f.index:
    f_indicator = f.loc[indicator]
    F_indicator = np.diag(f_indicator) @ LY_brics
    result.loc[indicator] = F_indicator
    
result

region                                                  OECD               \
sector                                                  Food     Clothing   
extension_name                                                              
CO2 emissions (unit: tonnes/year)              13,455,962.10 2,303,451.80   
Blue water consumption (unit: million m3/year)      5,292.04        20.17   
Employment (unit: 1000 people/year)                   756.22       155.91   

region                                                                      \
sector                                                Shelter Construction   
extension_name                                                               
CO2 emissions (unit: tonnes/year)              151,816,923.10 9,973,621.37   
Blue water consumption (unit: million m3/year)         236.41        10.58   
Employment (unit: 1000 people/year)                    487.11       529.75   

region                                                                \
sector                                         Manufactured products   
extension_name                                                         
CO2 emissions (unit: tonnes/year)                      76,411,303.88   
Blue water consumption (unit: million m3/year)                496.18   
Employment (unit: 1000 people/year)                         3,339.00   

region                                                                     \
sector                                              Mobility        Trade   
extension_name                                                              
CO2 emissions (unit: tonnes/year)              59,101,594.12 2,803,609.70   
Blue water consumption (unit: million m3/year)          0.00         0.00   
Employment (unit: 1000 people/year)                   634.73     2,591.88   

region                                                                BRICS  \
sector                                              Services           Food   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              11,051,575.01 320,602,092.64   
Blue water consumption (unit: million m3/year)          0.00     421,043.85   
Employment (unit: 1000 people/year)                 4,523.73     418,736.45   

region                                                        ...  \
sector                                              Clothing  ...   
extension_name                                                ...   
CO2 emissions (unit: tonnes/year)              40,804,311.97  ...   
Blue water consumption (unit: million m3/year)      1,144.21  ...   
Employment (unit: 1000 people/year)                20,765.75  ...   

region                                                                       \
sector                                                 Trade       Services   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              25,153,561.29 429,664,382.03   
Blue water consumption (unit: million m3/year)          0.00           0.00   
Employment (unit: 1000 people/year)                56,098.69     260,245.71   

region                                                   ROW                \
sector                                                  Food      Clothing   
extension_name                                                               
CO2 emissions (unit: tonnes/year)              31,455,485.05 12,875,544.11   
Blue water consumption (unit: million m3/year)     28,948.06        170.11   
Employment (unit: 1000 people/year)                41,678.20      1,041.91   

region                                                                       \
sector                                                Shelter  Construction   
extension_name                                                                
CO2 emissions (unit: tonnes/year)              556,389,702.25 50,490,479.26   
Blue water consumption

In [36]:
# Similarly we can do multiple regions/sectors for a single indicator in one line
result = f_co2 * LY.T  # ! the right side of * needs to be transposed if it contains multiple columns
result.T  # by transposing again we keep the same layout as we are used to

region                                                             OECD  \
final_demand_category        Final consumption expenditure by household   
region sector                                                             
OECD   Food                                              315,891,774.76   
       Clothing                                           40,588,698.99   
       Shelter                                         2,384,004,633.34   
       Construction                                       72,554,612.93   
       Manufactured products                             326,899,793.39   
       Mobility                                          908,023,922.11   
       Trade                                              30,440,269.88   
       Services                                          312,916,665.09   
BRICS  Food                                               16,977,773.11   
       Clothing                                           12,524,187.28   
       Shelter                                           455,042,025.70   
       Construction                                       44,934,914.36   
       Manufactured products                             211,099,107.65   
       Mobility                                           49,945,675.17   
       Trade                                               1,443,704.29   
       Services                                           11,223,456.31   
ROW    Food                                               56,088,178.44   
       Clothing                                           70,083,247.45   
       Shelter                                           574,071,896.19   
       Construction                                       32,962,977.03   
       Manufactured products                             124,991,466.52   
       Mobility                                          116,895,333.93   
       Trade                                               7,078,676.83   
       Services                                           18,986,586.66   

region                                                                \
final_demand_category        Final consumption expenditure by NPISHs   
region sector                                                          
OECD   Food                                             5,958,504.78   
       Clothing                                           384,181.94   
       Shelter                                        142,002,358.28   
       Construction                                    10,292,591.10   
       Manufactured products                           18,573,930.81   
       Mobility                                        57,842,280.95   
       Trade                                            2,241,572.87   
       Services                                        77,952,575.42   
BRICS  Food                                               564,828.06   
       Clothing                                           144,581.27   
       Shelter                                         21,295,602.58   
       Construction                                     2,327,935.85   
       Manufactured products                            9,869,356.20   
       Mobility                                         2,874,957.96   
       Trade                                               78,879.59   
       Services                                           774,199.60   
ROW    Food                                             1,667,032.01   
       Clothing                                           735,458.42   
       Shelter                                         33,922,463.13   
       Construction                                     2,364,567.61   
       Manufactured products                            6,623,361.82   
       Mobility                                         8,390,159.08   
       Trade                                              543,242.35   
       Services                                         3,076,506.85   

region                       

In [37]:
# the result is already broken down by components, which we can agggregate according to our needs
result.T.loc[:, 'BRICS'].sum(axis=1)  # = np.diag(f_co2) @ LY_brics

region  sector               
OECD    Food                       13,455,962.10
        Clothing                    2,303,451.80
        Shelter                   151,816,923.10
        Construction                9,973,621.37
        Manufactured products      76,411,303.88
        Mobility                   59,101,594.12
        Trade                       2,803,609.70
        Services                   11,051,575.01
BRICS   Food                      320,602,092.64
        Clothing                   40,804,311.97
        Shelter                 5,546,749,573.79
        Construction            2,261,010,412.06
        Manufactured products   1,957,439,674.97
        Mobility                  567,872,900.90
        Trade                      25,153,561.29
        Services                  429,664,382.03
ROW     Food                       31,455,485.05
        Clothing                   12,875,544.11
        Shelter                   556,389,702.25
        Construction               50,4

In [38]:
# here we aggregate by rows (i.e. by final demand category)
result.T.loc[:, 'BRICS'].sum(axis=0)

final_demand_category
Final consumption expenditure by household    3,960,358,509.29
Final consumption expenditure by NPISHs         685,406,224.59
Final consumption expenditure by government   1,348,960,727.60
Gross capital formation                       6,306,991,692.41
dtype: float64

In [39]:
# However f and Y can NOT be both matrix at the same time. One of them has to be a vector
# We have to choose which one to keep as DataFrame (f or Y) and aggregate the other in a pd.Series
f * (L @ Y)  

ValueError: cannot join with no overlapping index names

In [40]:
# Similarly, the following are equivalent
f @ L @ Y

region                                                                               OECD  \
final_demand_category                          Final consumption expenditure by household   
extension_name                                                                              
CO2 emissions (unit: tonnes/year)                                        6,195,669,577.43   
Blue water consumption (unit: million m3/year)                                 212,292.17   
Employment (unit: 1000 people/year)                                            428,340.15   

region                                                                                  \
final_demand_category                          Final consumption expenditure by NPISHs   
extension_name                                                                           
CO2 emissions (unit: tonnes/year)                                       410,501,128.55   
Blue water consumption (unit: million m3/year)                                5,311.55   
Employment (unit: 1000 people/year)                                          47,867.42   

region                                                                                      \
final_demand_category                          Final consumption expenditure by government   
extension_name                                                                               
CO2 emissions (unit: tonnes/year)                                           945,235,331.60   
Blue water consumption (unit: million m3/year)                                   11,373.63   
Employment (unit: 1000 people/year)                                              98,341.93   

region                                                                  \
final_demand_category                          Gross capital formation   
extension_name                                                           
CO2 emissions (unit: tonnes/year)                     2,940,608,644.04   
Blue water consumption (unit: million m3/year)               27,198.57   
Employment (unit: 1000 people/year)                         160,753.48   

region                                                                              BRICS  \
final_demand_category                          Final consumption expenditure by household   
extension_name                                                                              
CO2 emissions (unit: tonnes/year)                                        3,960,358,509.29   
Blue water consumption (unit: million m3/year)                                 351,554.33   
Employment (unit: 1000 people/year)                                            570,326.46   

region                                                                                  \
final_demand_category                          Final consumption expenditure by NPISHs   
extension_name                                                                           
CO2 emissions (unit: tonnes/year)                                       685,406,224.59   
Blue water consumption (unit: million m3/year)                               24,614.91   
Employment (unit: 1000 people/year)                                          86,400.41   

region                                                                                      \
final_demand_category                          Final consumption expenditure by government   
extension_name                                                                               
CO2 emissions (unit: tonnes/year)                                         1,348,960,727.60   
Blue water consumption (unit: million m3/year)                                   35,518.03   
Employment (unit: 1000 people/year)                                             138,548.55   

region                                                                  \
final_demand_category                          Gross capital formation   
extension_name                                                           
CO2

In [41]:
result = pd.DataFrame(
    data=np.nan,
    index=f.index,
    columns=Y.columns  # ! Y columns not index
)  

for indicator in f.index:
    f_indicator = f.loc[indicator]
    F_indicator = f_indicator * LY.T  # ! the right side of * needs to be transposed if it contains multiple columns
    F_indicator = F_indicator.T  # we transpose to keep the same layout as usual
    F_indicator_category = F_indicator.sum(axis=0)  # we aggregate by final demand category
    result.loc[indicator] = F_indicator_category.values

result

region                                                                               OECD  \
final_demand_category                          Final consumption expenditure by household   
extension_name                                                                              
CO2 emissions (unit: tonnes/year)                                        6,195,669,577.43   
Blue water consumption (unit: million m3/year)                                 212,292.17   
Employment (unit: 1000 people/year)                                            428,340.15   

region                                                                                  \
final_demand_category                          Final consumption expenditure by NPISHs   
extension_name                                                                           
CO2 emissions (unit: tonnes/year)                                       410,501,128.55   
Blue water consumption (unit: million m3/year)                                5,311.55   
Employment (unit: 1000 people/year)                                          47,867.42   

region                                                                                      \
final_demand_category                          Final consumption expenditure by government   
extension_name                                                                               
CO2 emissions (unit: tonnes/year)                                           945,235,331.60   
Blue water consumption (unit: million m3/year)                                   11,373.63   
Employment (unit: 1000 people/year)                                              98,341.93   

region                                                                  \
final_demand_category                          Gross capital formation   
extension_name                                                           
CO2 emissions (unit: tonnes/year)                     2,940,608,644.04   
Blue water consumption (unit: million m3/year)               27,198.57   
Employment (unit: 1000 people/year)                         160,753.48   

region                                                                              BRICS  \
final_demand_category                          Final consumption expenditure by household   
extension_name                                                                              
CO2 emissions (unit: tonnes/year)                                        3,960,358,509.29   
Blue water consumption (unit: million m3/year)                                 351,554.33   
Employment (unit: 1000 people/year)                                            570,326.46   

region                                                                                  \
final_demand_category                          Final consumption expenditure by NPISHs   
extension_name                                                                           
CO2 emissions (unit: tonnes/year)                                       685,406,224.59   
Blue water consumption (unit: million m3/year)                               24,614.91   
Employment (unit: 1000 people/year)                                          86,400.41   

region                                                                                      \
final_demand_category                          Final consumption expenditure by government   
extension_name                                                                               
CO2 emissions (unit: tonnes/year)                                         1,348,960,727.60   
Blue water consumption (unit: million m3/year)                                   35,518.03   
Employment (unit: 1000 people/year)                                             138,548.55   

region                                                                  \
final_demand_category                          Gross capital formation   
extension_name                                                           
CO2